<a href="https://colab.research.google.com/github/AliNajafpour/AI-Face-Identifier/blob/main/TextAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

In [13]:
import transformers
import os
from bs4 import BeautifulSoup
import requests

In [3]:
! git clone https://github.com/AliNajafpour/AI-Face-Identifier.git

Cloning into 'AI-Face-Identifier'...
remote: Enumerating objects: 130, done.
remote: Counting objects: 100% (130/130), done.
remote: Compressing objects: 100% (110/110), done.
remote: Total 130 (delta 57), reused 48 (delta 16), pack-reused 0 (from 0)
Receiving objects: 100% (130/130), 374.19 KiB | 11.00 MiB/s, done.
Resolving deltas: 100% (57/57), done.


In [4]:
question_answering_generator = transformers.pipeline('question-answering', model='deepset/roberta-base-squad2', device=0)
text2text_generator = transformers.pipeline("text2text-generation", model="google/flan-t5-base", device=0)
ner_generator = transformers.pipeline("ner", grouped_entities=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

Device set to use cuda:0
/usr/local/lib/python3.11/dist-packages/transformers/pipelines/token_classification.py:170: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="AggregationStrategy.SIMPLE"` instead.
  warnings.warn(


# Find the name with NER & text2text & question_answering

In [5]:
def extract_name_with_file(file, question):
  with open(file) as f:
    text = f.read()
  result1 = question_answering_generator(context=text, question=question)
  result2 = text2text_generator(f"who is main person in the text?.\n\n {text}")
  result3 = ner_generator(text)
  l = []
  for r in result3:
    if r['entity_group'] == 'PER':
      l.append(r['word'])
  return  result1,  result2,  max(set(l), key=l.count)


In [ ]:
name1, name2, name3 = extract_name_with_file('/content/AI-Face-Identifier/test/test_text/text1.txt', 'who is the main person in the texts?')
print('model 1 says: ' + name1['answer'], 'model 2 says: '  + name2[0]['generated_text'], 'model 3 says: ' + name3)

model 1 says: Ayatollah Ali Khamenei model 2 says: President Trump praises Puerto Rico’s response to Hurricane Maria as “an incredible, unsung success model 3 says: Whitehouse


In [ ]:
name1, name2, name3 = extract_name_with_file('/content/AI-Face-Identifier/test/test_text/text2.txt', 'who is the main person in the texts?')
print('model 1 says: ' + name1['answer'], 'model 2 says: '  + name2[0]['generated_text'], 'model 3 says: ' + name3)

model 1 says: Schumer model 2 says: Whitehouse says Republicans forced Democrats to choose between supporting a continuing resolution or a government shutdown. model 3 says: Whitehouse


In [ ]:
name1, name2, name3 = extract_name_with_file('/content/AI-Face-Identifier/test/test_text/text3.txt', 'who is the main person in the texts?')
print('model 1 says: ' + name1['answer'], 'model 2 says: '  + name2[0]['generated_text'], 'model 3 says: ' + name3)

model 1 says: Ayatollah Ali Khamenei model 2 says: Summary: China to continue retaliation against US tariffs model 3 says: Trump


In [ ]:
name1, name2, name3 = extract_name_with_file('/content/AI-Face-Identifier/test/test_text/text4.txt', 'who is the main person in the texts?')
print('model 1 says: ' + name1['answer'], 'model 2 says: '  + name2[0]['generated_text'], 'model 3 says: ' + name3)

model 1 says: President Donald Trump model 2 says: Trump model 3 says: Trump


In [ ]:
name1, name2, name3 = extract_name_with_file('/content/AI-Face-Identifier/test/test_text/text5.txt', 'who is the main person in the texts?')
print('model 1 says: ' + name1['answer'], 'model 2 says: '  + name2[0]['generated_text'], 'model 3 says: ' + name3)

model 1 says: Kushner model 2 says: Donald Trump model 3 says: Trump


# Find the name with NER throu all texts

In [6]:
def extract_name_NER(folder_path):
  names = []
  for path in os.listdir(folder_path):
    if path.lower().endswith('.txt'):
        full_path = os.path.join(folder_path, path)
        with open(full_path) as f:
          text = f.read()
        result = ner_generator(text)
        file_names = []
        for r in result:
            if r['entity_group'] == 'PER':
              names.append(r['word'])
  return max(set(names), key=names.count)

In [ ]:
extract_name_NER('/content/AI-Face-Identifier/test/test_text')

'Trump'

In [8]:
text1 = 'William Bradley Pitt (born December 18, 1963) is an American actor and film producer'
text2 = 'Donald John Trump (born June 14, 1946) is an American politician, media personality, and businessman who is the 47th president of the United States'
text3 = 'Cristiano Ronaldo dos Santos Aveiro (Portuguese pronunciation: [kɾiʃˈtjɐnu ʁɔˈnaldu] ⓘ; born 5 February 1985) is a Portuguese professional footballer who plays as a forward for and captains both Saudi Pro League club Al-Nassr and the Portugal national team'

In [9]:
result = ner_generator(text1)
result

[{'entity_group': 'PER',
  'score': np.float32(0.9994767),
  'word': 'William Bradley Pitt',
  'start': 0,
  'end': 20},
 {'entity_group': 'MISC',
  'score': np.float32(0.9970667),
  'word': 'American',
  'start': 52,
  'end': 60}]

In [10]:
result = ner_generator(text2)
result

[{'entity_group': 'PER',
  'score': np.float32(0.99942666),
  'word': 'Donald John Trump',
  'start': 0,
  'end': 17},
 {'entity_group': 'MISC',
  'score': np.float32(0.998502),
  'word': 'American',
  'start': 45,
  'end': 53},
 {'entity_group': 'LOC',
  'score': np.float32(0.998502),
  'word': 'United States',
  'start': 134,
  'end': 147}]

In [11]:
result = ner_generator(text3)
result

[{'entity_group': 'PER',
  'score': np.float32(0.9677963),
  'word': 'Cristiano Ronaldo dos Santos Aveiro',
  'start': 0,
  'end': 35},
 {'entity_group': 'MISC',
  'score': np.float32(0.9572215),
  'word': 'Portuguese',
  'start': 37,
  'end': 47},
 {'entity_group': 'PER',
  'score': np.float32(0.7615413),
  'word': 'kɾiʃ',
  'start': 64,
  'end': 68},
 {'entity_group': 'PER',
  'score': np.float32(0.6608038),
  'word': '##tj',
  'start': 69,
  'end': 71},
 {'entity_group': 'PER',
  'score': np.float32(0.94400144),
  'word': '##nu ʁ',
  'start': 72,
  'end': 76},
 {'entity_group': 'PER',
  'score': np.float32(0.64250016),
  'word': '##du',
  'start': 81,
  'end': 83},
 {'entity_group': 'MISC',
  'score': np.float32(0.9970181),
  'word': 'Portuguese',
  'start': 115,
  'end': 125},
 {'entity_group': 'MISC',
  'score': np.float32(0.94633836),
  'word': 'Saudi Pro League',
  'start': 195,
  'end': 211},
 {'entity_group': 'ORG',
  'score': np.float32(0.907986),
  'word': 'Al - Nassr',
  's

In [65]:
def analysis_wikipedia_with_NER(wikipedia_url):
  headers = {'User-Agent': 'Mozilla/5.0'}
  response = requests.get(wikipedia_url, headers=headers)
  soup = BeautifulSoup(response.text, 'html.parser')

  data = soup.find_all('p')
  for p in data:
    if len(p.get_text()) > 20:
      result = ner_generator(p.get_text())
      for r in result:
        if r['entity_group'] == 'PER':
          name = r['word']
          result = question_answering_generator(context=p.get_text(), question=f'What is {name} nationality?')
          result2 = question_answering_generator(context=p.get_text(), question=f'What is {name} occupation?')
          return  f"{name} is {result['answer']} and he/she is {result2['answer']}"
        else:
          return 'no name was found'


In [66]:
analysis_wikipedia_with_NER('https://en.wikipedia.org/wiki/Donald_Trump')

'Donald John Trump is American and he/she is businessman'

In [67]:
analysis_wikipedia_with_NER('https://en.wikipedia.org/wiki/Jafar_Panahi')

'Jafar Panâhi is Iranian and he/she is Iranian film director, screenwriter, and editor'

In [68]:
analysis_wikipedia_with_NER('https://en.wikipedia.org/wiki/Cristiano_Ronaldo')

'Cristiano Ronaldo dos Santos Aveiro is Portuguese and he/she is forward'

In [69]:
analysis_wikipedia_with_NER('https://en.wikipedia.org/wiki/Michele_Abruzzo')

'Michele Abruzzo is Italian and he/she is actor'